<h1> Pesquisa em bancos de dados para astronomia</h1>

Primeiramente e necessario que o usuario tenha instalado no seu computador os modulos: numpy, pandas, astroquery, astropy, seaborn e matplotlib. O modulo Astrofunctions e um documento .py , criado pelo Rafael, que precisa estar alocado no mesmo diretorio que o codigo sera colocado.
Para instalacao dos modulos extras e necessario abrir o Terminal e digitar o comando pip install O_nome_do_modulo_de_interesse. O comando e o mesmo para usuarios do mac e para usuarios da distribuicao do anaconda no windows.  

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import astropy.units as u

from astroquery.ned import Ned
from astroquery.simbad import Simbad
from astropy.table import Table,QTable
from astropy import coordinates
from astropy.coordinates import SkyCoord

#Modulos autorais
import AstroFunctions as astro
from UnitConversion import hmstodegree, dmstodegree


Utilizando o modulo Pandas, o proximo passo e ler um arquivo csv alocado no endereco entre 
(""). Apos a leitura desse arquivo, o comando vvds[].values serve apara alocar em uma array 
os valores contidos na coluna intitulada "RA"

In [8]:
vvds = pd.read_csv("vvds-not.csv")
RA = vvds["RA"].values
DEC = vvds["DEC"].values

Para que seja possivel percorrer varias regioes relacionadas aos varios valores contidos na coluna que descreve as coordenadas e necessario realizar a pesquisa com o auxilio de um for que percorre todas as linhas das arrays e pesquisa regiao por regiao.

In [9]:
co = coordinates.SkyCoord(ra = RA[0],dec= DEC[0],unit=(u.deg,u.deg))
#Pesquisa no banco de dados NED
Ned_table_um = Ned.query_region(co, radius=0.0007 * u.deg)
#Pesquisa no banco de dados SIMBAD
Simbad_table_um = Simbad.query_region(co, radius=0.0007 * u.deg)

In [10]:
Ned_table_um

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,bytes30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,VVDS 020086348,36.6795,-4.821,G,119018.0,0.397,,23.6V,0.002,4,0,3,2,1,0,0
2,VVDS 020086179,36.67985,-4.82053,VisS,--,--,,24.8R,0.036,1,0,0,1,0,0,0
3,VVDS 020086136,36.68013,-4.82081,VisS,--,--,,24.7R,0.039,1,0,0,1,0,0,0


In [11]:
dmstodegree(['-04 49 15.7'])

[-4.821027777777777]


Com relação ao astroquery, ele possui um tempo limite de conexão com o servidor que e pequeno, por isso para uma pesquisa de muitos dados e necessário alterar o valor da variável timeout na pasta em que o modulo astroquery foi instalado e na respectiva pasta do banco de dados.

In [6]:
l1 = []
l2 = []
for i in range(len(RA)):
    co = coordinates.SkyCoord(ra = RA[i],dec= DEC[i],unit=(u.deg,u.deg))
    #Pesquisa no banco de dados NED
    ned_table = Ned.query_region(co, radius=0.001* u.deg) #sugestao 0.0007
    try:
        df = ned_table.to_pandas()
        l1.append(df)
    except:
        pass
    #Pesquisa no banco de dados SIMBAD
    simbad_table = Simbad.query_region(co, radius=0.0007* u.deg)
    try:
        df = simbad_table.to_pandas()
        l2.append(df)
    except:   
        pass

KeyboardInterrupt: 

In [ ]:
l1 = []
l2 = []
for i in range(len(RA)):
    co = coordinates.SkyCoord(ra = RA[i],dec= DEC[i],unit=(u.deg,u.deg))
    #Pesquisa no banco de dados NED
    ned_table = Ned.query_region(co, radius=0.0007* u.deg) #sugestao 0.0007
    df = ned_table.to_pandas()
    l1.append(df)
    #Pesquisa no banco de dados SIMBAD
    simbad_table = Simbad.query_region(co, radius=0.0007* u.deg)
    df = simbad_table.to_pandas()
    l2.append(df)


In [ ]:
ned = pd.concat(l1, axis = 0, ignore_index = True)    
simbad = pd.concat(l2, axis=0, ignore_index = True)

In [ ]:
simbad_convertido = simbad.replace(simbad["RA"].values ,hmstodegree(simbad["RA"].values))
simbad_convertido = simbad_convertido.replace(simbad_convertido["DEC"].values ,dmstodegree(simbad["DEC"].values))

In [ ]:
Coluna_RA = simbad["RA"].values
Coluna_DEC = simbad["DEC"].values


In [ ]:
Coluna_DEC 

In [ ]:
print(dmstodegree(['0 53 17.016']))
print(hmstodegree(['0 39 15.9']))


In [ ]:
coordenadas = SkyCoord('0h39m15.9s','-04d49m15.7s', frame = 'icrs' )
print(Coluna_RA[0],Coluna_DEC[0])
print(coordenadas)

In [ ]:
ned_table

In [ ]:
ned

In [ ]:
ned_table = Table.from_pandas(ned)
simbad_table = Table.from_pandas(simbad_convertido)

In [ ]:
error = 0.1
data = astro.match(ned_table, simbad_table,'RA','RA','DEC','DEC',error)
if len(data) != 0:
    data.add_column(1.5, name = 'random')
    for i in range(0, len(data)):
        data[i]['random'] = np.random.random()

In [ ]:
data

In [ ]:
# Criando o ambiente do gráfico 
sns.set_style("white")
plt.figure(figsize=(10, 10))

# Gráfico de Dispersão
RA_DEC_Ned =ned_table["RA","DEC"]
Ned_pandas = RA_DEC_Ned.to_pandas()
RA_DEC_Simbad =simbad_table["RA","DEC"]
Simbad_pandas = RA_DEC_Simbad.to_pandas()
RA_DEC_Data = data["RA_1","DEC_1"]
Data_pandas = RA_DEC_Data.to_pandas()
g = sns.scatterplot(x = "RA", y = "DEC", data = Ned_pandas, label = 'Ned')
h = sns.scatterplot(x = "RA", y = "DEC", data = Simbad_pandas, label = 'Simbad')
w = sns.scatterplot(x = "RA_1", y = "DEC_1", data= Data_pandas, label = 'Cross Match')
plt.legend()
plt.show()

In [ ]:
g = sns.scatterplot(x = "RA", y = "DEC", data = Ned_pandas, label = 'Ned')
h = sns.scatterplot(x = "RA", y = "DEC", data = Simbad_pandas, label = 'Simbad')
w = sns.scatterplot(x = "RA_1", y = "DEC_1", data= Data_pandas, label = 'Cross Match')
plt.legend()
plt.xlim(52.5,54)
plt.ylim(-29,-25)
plt.show()

In [ ]:
g = sns.scatterplot(x = "RA", y = "DEC", data = Ned_pandas, label = 'Ned')
h = sns.scatterplot(x = "RA", y = "DEC", data = Simbad_pandas, label = 'Simbad')
w = sns.scatterplot(x = "RA_1", y = "DEC_1", data= Data_pandas, label = 'Cross Match')
plt.legend()
plt.xlim(53.2,53.6)
plt.ylim(-27.8,-28.1)
plt.show()

In [ ]:
error = 0.005
data = astro.match(ned_table, simbad_table,'RA','RA','DEC','DEC',error)
if len(data) != 0:
    data.add_column(1.5, name = 'random')
    for i in range(0, len(data)):
        data[i]['random'] = np.random.random()

In [ ]:
data

In [ ]:
RA_DEC_Data = data["RA_1","DEC_1"]
Data_pandas = RA_DEC_Data.to_pandas()
g = sns.scatterplot(x = "RA", y = "DEC", data = Ned_pandas, label = 'Ned')
h = sns.scatterplot(x = "RA", y = "DEC", data = Simbad_pandas, label = 'Simbad')
w = sns.scatterplot(x = "RA_1", y = "DEC_1", data= Data_pandas, label = 'Cross Match')
plt.legend()
plt.show()

In [ ]:
g = sns.scatterplot(x = "RA", y = "DEC", data = Ned_pandas, label = 'Ned')
h = sns.scatterplot(x = "RA", y = "DEC", data = Simbad_pandas, label = 'Simbad')
w = sns.scatterplot(x = "RA_1", y = "DEC_1", data= Data_pandas, label = 'Cross Match')
plt.legend()
plt.xlim(53.3,53.6)
plt.ylim(-27.8,-28.2)
plt.show()